In [1]:
import os
import sys
import random
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch import optim
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from tqdm import tqdm, trange
from collections import defaultdict

RDKit WARNING: [13:57:54] Enabling RDKit 2019.09.1 jupyter extensions


In [2]:
os.chdir('/home/baba/Downloads/retrosynthesis_planner-master/')


In [3]:
def fps_to_arr(fps):
    """Faster conversion to ndarray"""
    arrs = []
    for fp in fps:
        onbits = list(fp.GetOnBits())
        arr = np.zeros(fp.GetNumBits())
        arr[onbits] = 1
        arrs.append(arr)
    arrs = np.array(arrs)
    return arrs

In [4]:
class Highway(nn.Module):
    def __init__(self,size, num_layers, f):
        super(Highway, self).__init__()
        self.num_layers = num_layers
        self.nonlinear = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.linear = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.gate = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.f = f
    def forward(self,x):
        for layer in range(self.num_layers):
            gate = torch.sigmoid(self.gate[layer](x))
            nonlinear = self.f(self.nonlinear[layer](x))
            linear = self.linear[layer](x)
            x = gate * nonlinear + (1- gate) * linear
        return x

In [5]:
def fingerprint_mols(mols, fp_dim):
    fps = []
    for mol in mols:
        mol = Chem.MolFromSmiles(mol)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=int(fp_dim))
        fps.append(fp)
    return fps

In [6]:
def fingerprint_reactions(reactions, fp_dim):
    fps = []
    for r in reactions:
        rxn = AllChem.ReactionFromSmarts(r)
        fp = AllChem.CreateStructuralFingerprintForReaction(rxn)
        fold_factor = fp.GetNumBits()//fp_dim
        fp = DataStructs.FoldFingerprint(fp, fold_factor)
        fps.append(fp)
    return fps

In [7]:
class RolloutPolicyNet(nn.Module):
    def __init__(self, n_rules, fp_dim=8192,k=10):
        super().__init__()
        self.fp_dim = fp_dim
        self.n_rules = n_rules
        self.linear1 = nn.Linear(self.fp_dim, 512)
        self.dropout = nn.Dropout(0.4)
        self.linear2 = nn.Linear(512, n_rules)
    def forward(self, x):
        x = F.elu(self.linear1(x))
        x = self.dropout(x)
        x = F.softmax(self.linear2(x))
        return x

    def preprocess(self, X):
        X = fingerprint_mols(X, self.fp_dim)
        X = fps_to_arr(X)
        X =torch.from_numpy(X).float()
        return X
    

In [8]:
class InScopeFilterNet(nn.Module):
    def __init__(self, product_fp_dim=16384, reaction_fp_dim=2048):
        super().__init__()
        self.prod_fp_dim = product_fp_dim
        self.react_fp_dim = reaction_fp_dim
        #product branch
        self.prod_l1 = nn.Linear(self.prod_fp_dim,1024)
        self.dropout = nn.Dropout(0.3)
        self.highway = Highway(1024, 5, f=F.elu)
        #reaction branch
        self.react_layer = nn.Linear(2048, 1024)
    def forward(self,x):
        self.x = x
        self.x_prod = self.x[:,:self.prod_fp_dim]
        self.x_react = self.x[:,self.prod_fp_dim:]
        prod_x = self.dropout(F.elu(self.prod_l1(self.x_prod)))
        prod_x = self.highway(prod_x)
        react_x = F.elu(self.react_layer(self.x_react))
        prod_norm = F.normalize(prod_x,dim=0,p=2)
        react_norm = F.normalize(react_x,dim=0,p=2)
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        cosine_sim = cos(prod_norm, react_norm)
        out = torch.sigmoid(cosine_sim)
#         print('inscopefileter output:::',out)
        return out
        
        

    def preprocess(self, X):
        # Compute fingerprints
        prod_mols, react_mols = zip(*X)
        prod_fps = fingerprint_mols(prod_mols, self.prod_fp_dim)
        react_fps = fingerprint_reactions(react_mols, self.react_fp_dim)
        return torch.from_numpy(np.hstack([prod_fps, react_fps])).float()


In [9]:
if __name__ == '__main__':
    print('Loading data...')
    prod_to_rules = defaultdict(set)
    with open('data/templates.dat', 'r') as f:
        for l in tqdm(f, desc='products'):
            rule, prod = l.strip().split('\t')
#             print('rule',rule,'prod',prod)
            prod_to_rules[prod].add(rule)
    
    rollout_rules = {}
    with open('data/rollout.dat', 'r') as f:
        for i, l in tqdm(enumerate(f), desc='rollout'): 
            rule = l.strip()
            rollout_rules[rule] = i
            
    expansion_rules = {}
    with open('data/expansion.dat', 'r') as f:
        for i, l in tqdm(enumerate(f), desc='expansion'):
            rule = l.strip()
            expansion_rules[rule] = i

    prods = list(prod_to_rules.keys())
    exp_fp_dim = 1e4

    def chunker(seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))

    def reducefn(a, b):

        n_a, mean_a, var_a = a
        n_b, mean_b, var_b = b
        n_ab = n_a + n_b
        mean_ab = ((mean_a * n_a) + (mean_b * n_b)) / n_ab
        var_ab = (((n_a * var_a) + (n_b * var_b)) / n_ab) + ((n_a * n_b) * ((mean_b - mean_a) / n_ab)**2)
        return n_ab, mean_ab, var_ab

    def mapfn(chunk):
        chunk = fingerprint_mols(chunk, exp_fp_dim)
        arrs = np.log(fps_to_arr(chunk) + 1)
        return len(arrs), np.mean(arrs, axis=0), np.var(arrs, axis=0)

    save_path = 'model'
    ckpt_path = os.path.join(save_path, 'model.ckpt')
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    from multiprocessing import Pool
    from functools import reduce
    chunk_size = 100
    prods = []
    for prod, rules in prod_to_rules.items():
        if any(r in expansion_rules for r in rules):
            prods.append(prod)
    # prods = random.sample(prods, 200000)
    with Pool() as p:
        _, _, var = reduce(reducefn, tqdm(p.imap(mapfn, chunker(prods, chunk_size)), total=len(prods)//chunk_size))
    idx = np.where(var > 0)[0]


products: 356it [00:00, 2235.99it/s]

Loading data...


products: 1000001it [00:06, 156827.07it/s]
rollout: 1415it [00:00, 84717.09it/s]
expansion: 43637it [00:00, 208662.49it/s]
443it [00:22, 19.92it/s]                         


In [10]:
rollout = RolloutPolicyNet(n_rules=len(rollout_rules))


In [11]:
filter = InScopeFilterNet()

In [36]:
rollout

RolloutPolicyNet(
  (linear1): Linear(in_features=8192, out_features=512, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (linear2): Linear(in_features=512, out_features=1415, bias=True)
)

In [13]:
filter

filter

In [47]:
expansion

ExpansionPolicyNet(
  (nonlinear): ModuleList(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
  )
  (linear): ModuleList(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
  )
  (gate): ModuleList(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512

In [12]:
class ExpansionPolicyNet(nn.Module):
    def __init__(self, idx, n_rules, fp_dim=1e6):
        super().__init__()
        self.idx = idx
        self.fp_dim = fp_dim
        self.n_rules = n_rules
        self.layer1 = nn.Linear(int(self.fp_dim), 512)
        self.highway = Highway(512,5, f=F.elu)
        self.out = nn.Linear(512,n_rules)
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.1)
    def forward(self,x):
#         print('inforward',type(x))
        x = F.elu(self.layer1(x)) 
        x = self.dropout1(x)
        x = F.elu(self.highway(x))
        x = self.dropout2(x)
        x = self.out(x)
        x = F.softmax(x)
        return x

    def preprocess(self, X):
        X = fingerprint_mols(X, self.fp_dim)
        X = fps_to_arr(X)
        X =torch.from_numpy(X).float()
#         print('type in preprocess',type(X))
        
        return X

In [13]:
expansion = ExpansionPolicyNet(idx, n_rules=len(expansion_rules), fp_dim=exp_fp_dim)

In [18]:
def train(net,X,y,batch_size=16,epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(),lr=0.0001)
    
    it = trange(epochs)
    n_steps = int(np.ceil(len(X)/batch_size))
    for e in it:
        total = 0
        running_loss = 0
        correct = 0
        xy = list(zip(X, y))
        random.shuffle(xy)
        X, y = zip(*xy)
        for i in tqdm(range(n_steps)):
            l = i*batch_size
            u = l + batch_size
            X_batch, y_batch = X[l:u], y[l:u]
            X_batch = net.preprocess(X_batch)
            optimizer.zero_grad()
            outputs = net.forward(X_batch)
            y_batch = np.array(y_batch)
            y_batch = torch.from_numpy(y_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()
            _, predicted = torch.max(outputs.data, 1)
#             print('Intrain::::',_,"predicted:::",predicted)
            total += y_batch.size(0)
#             print('y_batch',y_batch)
            correct += (predicted == y_batch).sum().item()
            print('Accuracy of the network: %d %%' % (100 * correct / total))
            print('training loss:',(running_loss/n_steps))

In [34]:
print('Expansion training...')
X, y = [], []
for prod, rules in tqdm(prod_to_rules.items(), desc='data prep'):
    rules = [r for r in rules if r in expansion_rules]
    if not rules: continue

        # Ideally trained as multilabel,
        # but multiclass, single label is easier atm
    for r in rules:
        id = expansion_rules[r]
        y.append(id)
        X.append(prod)
#         print('type in expansion training::',type(X))
print('Training size:', len(X))
train(expansion, np.array(X), np.array(y), batch_size=256, epochs=10)












data prep:   0%|          | 0/561244 [00:00<?, ?it/s]










data prep:   6%|▋         | 36299/561244 [00:00<00:01, 362976.50it/s]

Expansion training...













data prep:  13%|█▎        | 70642/561244 [00:00<00:01, 356880.83it/s]










data prep:  20%|██        | 114779/561244 [00:00<00:01, 378624.11it/s]










data prep:  29%|██▉       | 162317/561244 [00:00<00:00, 403246.25it/s]










data prep:  38%|███▊      | 211885/561244 [00:00<00:00, 427141.43it/s]










data prep:  46%|████▌     | 259270/561244 [00:00<00:00, 440157.19it/s]










data prep:  55%|█████▍    | 305918/561244 [00:00<00:00, 447735.63it/s]










data prep:  63%|██████▎   | 356376/561244 [00:00<00:00, 463395.07it/s]










data prep:  73%|███████▎  | 406945/561244 [00:00<00:00, 475321.05it/s]










data prep:  83%|████████▎ | 465105/561244 [00:01<00:00, 502888.67it/s]










data prep: 100%|██████████| 561244/561244 [00:01<00:00, 480132.75it/s]

Training size: 45653


NameError: name 'expansion' is not defined

In [38]:

# Rollout training
print('Rollout training...')
X, y = [], []
for prod, rules in tqdm(prod_to_rules.items(), desc='data prep'):
    rules = [r for r in rules if r in rollout_rules]
    if not rules: continue

    # Ideally trained as multilabel,
    # but multiclass, single label is easier atm
    for r in rules:
        id = rollout_rules[r]
        y.append(id)
        X.append(prod)
print('Training size:', len(X))
train(rollout, np.array(X), np.array(y), batch_size=256, epochs=10)












data prep:   0%|          | 0/561244 [00:00<?, ?it/s]










data prep:   9%|▉         | 49259/561244 [00:00<00:01, 492585.77it/s]

Rollout training...













data prep:  18%|█▊        | 101835/561244 [00:00<00:00, 502089.78it/s]










data prep:  28%|██▊       | 155253/561244 [00:00<00:00, 511302.97it/s]










data prep:  39%|███▉      | 219680/561244 [00:00<00:00, 545047.01it/s]










data prep:  49%|████▉     | 275422/561244 [00:00<00:00, 548699.45it/s]










data prep:  60%|██████    | 337553/561244 [00:00<00:00, 568635.26it/s]










data prep:  71%|███████   | 399734/561244 [00:00<00:00, 583605.41it/s]










data prep:  82%|████████▏ | 460226/561244 [00:00<00:00, 589838.09it/s]










data prep: 100%|██████████| 561244/561244 [00:00<00:00, 571880.15it/s]











  0%|          | 0/10 [00:00<?, ?it/s]











  0%|          | 0/6 [00:00<?, ?it/s]

Training size: 1525


/home/baba/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.












 17%|█▋        | 1/6 [00:00<00:01,  2.59it/s]

...

Intrain:::: tensor([0.0010, 0.0008, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.0008,
        0.0008, 0.0013, 0.0009, 0.0009, 0.0010, 0.0009, 0.0012, 0.0008, 0.0008,
        0.0008, 0.0010, 0.0011, 0.0008, 0.0008, 0.0011, 0.0008, 0.0009, 0.0008,
        0.0008, 0.0008, 0.0008, 0.0008, 0.0011, 0.0010, 0.0008, 0.0008, 0.0008,
        0.0009, 0.0008, 0.0008, 0.0009, 0.0009, 0.0009, 0.0010, 0.0008, 0.0008,
        0.0008, 0.0008, 0.0010, 0.0008, 0.0009, 0.0009, 0.0008, 0.0008, 0.0009,
        0.0010, 0.0008, 0.0008, 0.0008, 0.0009, 0.0009, 0.0009, 0.0008, 0.0008,
        0.0008, 0.0009, 0.0009, 0.0009, 0.0010, 0.0009, 0.0008, 0.0008, 0.0008,
        0.0008, 0.0009, 0.0008, 0.0008, 0.0008, 0.0009, 0.0008, 0.0009, 0.0008,
        0.0010, 0.0009, 0.0010, 0.0008, 0.0008, 0.0008, 0.0010, 0.0008, 0.0009,
        0.0009, 0.0008, 0.0008, 0.0011, 0.0008, 0.0009, 0.0008, 0.0008, 0.0008,
        0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0009, 0.0009, 0.0010, 0.0009,
        0.0008, 0.0010, 0.00













 83%|████████▎ | 5/6 [00:02<00:00,  2.50it/s]

Intrain:::: tensor([0.0008, 0.0008, 0.0016, 0.0008, 0.0009, 0.0008, 0.0009, 0.0008, 0.0009,
        0.0009, 0.0009, 0.0009, 0.0012, 0.0008, 0.0009, 0.0008, 0.0009, 0.0009,
        0.0008, 0.0008, 0.0009, 0.0012, 0.0008, 0.0009, 0.0010, 0.0008, 0.0009,
        0.0009, 0.0009, 0.0011, 0.0008, 0.0008, 0.0009, 0.0009, 0.0009, 0.0008,
        0.0008, 0.0009, 0.0008, 0.0008, 0.0008, 0.0009, 0.0008, 0.0008, 0.0008,
        0.0009, 0.0009, 0.0010, 0.0009, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008,
        0.0008, 0.0008, 0.0008, 0.0010, 0.0008, 0.0008, 0.0009, 0.0008, 0.0008,
        0.0011, 0.0008, 0.0010, 0.0008, 0.0008, 0.0008, 0.0009, 0.0009, 0.0009,
        0.0008, 0.0008, 0.0012, 0.0010, 0.0008, 0.0008, 0.0008, 0.0008, 0.0009,
        0.0008, 0.0009, 0.0009, 0.0009, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008,
        0.0011, 0.0009, 0.0011, 0.0008, 0.0008, 0.0009, 0.0009, 0.0009, 0.0010,
        0.0010, 0.0008, 0.0009, 0.0008, 0.0011, 0.0008, 0.0008, 0.0008, 0.0008,
        0.0008, 0.0009, 0.00













100%|██████████| 6/6 [00:02<00:00,  2.51it/s]











100%|██████████| 10/10 [00:25<00:00,  2.51s/it]

Intrain:::: tensor([0.0009, 0.0012, 0.0009, 0.0008, 0.0009, 0.0009, 0.0010, 0.0009, 0.0010,
        0.0010, 0.0009, 0.0009, 0.0009, 0.0009, 0.0008, 0.0008, 0.0009, 0.0008,
        0.0010, 0.0010, 0.0008, 0.0009, 0.0009, 0.0009, 0.0008, 0.0008, 0.0010,
        0.0008, 0.0008, 0.0012, 0.0010, 0.0009, 0.0008, 0.0008, 0.0010, 0.0009,
        0.0008, 0.0010, 0.0008, 0.0008, 0.0008, 0.0009, 0.0008, 0.0015, 0.0013,
        0.0009, 0.0008, 0.0009, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0009,
        0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0009, 0.0008, 0.0011, 0.0010,
        0.0008, 0.0010, 0.0009, 0.0008, 0.0009, 0.0013, 0.0008, 0.0008, 0.0008,
        0.0008, 0.0008, 0.0008, 0.0008, 0.0012, 0.0010, 0.0008, 0.0009, 0.0008,
        0.0009, 0.0008, 0.0008, 0.0009, 0.0009, 0.0008, 0.0008, 0.0008, 0.0009,
        0.0008, 0.0008, 0.0009, 0.0009, 0.0008, 0.0010, 0.0010, 0.0009, 0.0008,
        0.0008, 0.0009, 0.0008, 0.0009, 0.0009, 0.0008, 0.0008, 0.0008, 0.0009,
        0.0010, 0.0009, 0.00

In [19]:
def train_inscope(net,X,y,batch_size=16,epochs=10):
    criterion = nn.BCELoss()    
    optimizer = optim.Adam(net.parameters(),lr=0.0001)    
    it = trange(epochs)
    n_steps = int(np.ceil(len(X)/batch_size))
    for e in it:
        total = 0
        running_loss = 0
        correct = 0
        xy = list(zip(X, y))
        random.shuffle(xy)
        X, y = zip(*xy)
        for i in tqdm(range(n_steps)):
            l = i*batch_size
            u = l + batch_size
            X_batch, y_batch = X[l:u], y[l:u]
            X_batch = net.preprocess(X_batch)
            optimizer.zero_grad()
            outputs = net.forward(X_batch)
            y_batch = np.array(y_batch)
            y_batch = torch.from_numpy(y_batch).float()
#             print('y_batch in trainscope::',y_batch)
            loss = criterion(torch.sigmoid(outputs), y_batch)
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()
            print('training loss:',running_loss)

In [65]:
print('In-Scope Filter training...')
X, y = [], []
exists = set()
for prod, rules in tqdm(prod_to_rules.items(), desc='data prep'):
    rules = [r for r in rules if r in expansion_rules]
    if not rules: continue

    for r in rules:
        y.append(1.)
        X.append((prod, r))
        exists.add('{}_{}'.format(prod, r))

# Generate negative examples
target_size = len(X) * 2
pbar = tqdm(total=target_size//2, desc='data prep (negative)')
prods = list(prod_to_rules.keys())
exprules = list(expansion_rules.keys())
while len(X) < target_size:
    prod = random.choice(prods)
    rule = random.choice(exprules)
    key = '{}_{}'.format(prod, r)
    if key in exists:
        continue
    else:
        y.append(0.)
        X.append((prod, rule))
        pbar.update(1)
pbar.close()
print('Training size:', len(X))
train_inscope(filter, X, y, batch_size=512, epochs=3)



































data prep:   0%|          | 0/561244 [00:00<?, ?it/s]
































data prep:   7%|▋         | 40564/561244 [00:00<00:01, 405636.52it/s]

In-Scope Filter training...



































data prep:  15%|█▍        | 82528/561244 [00:00<00:01, 409737.12it/s]
































data prep:  20%|██        | 115025/561244 [00:00<00:01, 379997.58it/s]
































data prep:  28%|██▊       | 158910/561244 [00:00<00:01, 395926.26it/s]
































data prep:  34%|███▍      | 193440/561244 [00:00<00:00, 379240.96it/s]
































data prep:  43%|████▎     | 242900/561244 [00:00<00:00, 407772.93it/s]
































data prep:  52%|█████▏    | 294270/561244 [00:00<00:00, 434657.53it/s]
































data prep:  62%|██████▏   | 348417/561244 [00:00<00:00, 461995.49it/s]
































data prep:  72%|███████▏  | 402963/561244 [00:00<00:00, 484223.00it/s]
































data prep:  82%|████████▏ | 459936/561244 [00:01<00:00, 507051.58it/s]
































data prep: 100%|██████████| 561244/561244

Training size: 91306




































  0%|          | 0/179 [00:00<?, ?it/s]/home/baba/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


































  1%|          | 1/179 [00:31<1:33:09, 31.40s/it]

...



































  1%|          | 2/179 [01:03<1:32:22, 31.32s/it]

training loss: 1.360697329044342




































  2%|▏         | 3/179 [01:35<1:32:49, 31.64s/it]

training loss: 2.0331393480300903


KeyboardInterrupt: 

In [ ]:
"""
MCTS
"""

In [14]:
import math
from tqdm import tqdm
from time import time

EXPLORE_PARAM = 2

In [15]:
class Node:
    def __init__(self, state, parent=None, action=None, is_terminal=False):
        self.state = state
        self.children = []
        self.parent = parent
        self.n_visits = 0
        self.reward = 0
        self.action = action
        self.is_terminal = is_terminal

    @property
    def value(self):
        """UCB1"""
        if self.n_visits == 0:
            return float('inf')
        return self.reward/self.n_visits + \
            EXPLORE_PARAM*math.sqrt(math.log(self.parent.n_visits)/self.n_visits)

    def best_child(self):
        return max(self.children, key=lambda n: n.value)


In [16]:
def mcts(root, expansion_policy, rollout_policy, iterations=2000, max_depth=200):
    """
    Monte Carlo Tree Search
    - `expansion_policy` should be a function that takes a node and returns a
    list of child nodes
    - `rollout_policy` should be a function that takes a node and returns a
    reward for that node
    """
    root.children = expansion_policy(root)
    print('root.children:-',root.children)

    # MCTS
    for _ in tqdm(range(iterations)):
        cur_node = root

        # Selection
        while True:
            if cur_node.n_visits > 0 and cur_node.children:
                cur_node = cur_node.best_child()
            else:
                break

        if cur_node.n_visits > 0:
            # If selection took us to a terminal node,
            # this seems to be the best path
            if cur_node.is_terminal:
                break

            # Expansion
            s = time()
            cur_node.children = expansion_policy(cur_node)
            print('Expansion took:', time() - s)
            cur_node = cur_node.best_child()

        # Rollout
        s = time()
        reward = rollout_policy(cur_node, max_depth=max_depth)
        print('reward:-',reward)
        print('Rollout took:', time() - s)

        # Update
        cur_node.reward += reward
        cur_node.n_visits += 1
        parent = cur_node.parent
        while parent is not None:
            parent.reward += reward
            parent.n_visits += 1
            parent = parent.parent

    # Return best path
    cur_node = root
    path = [cur_node]
    for _ in range(max_depth):
        cur_node = cur_node.best_child()
        path.append(cur_node)
        if cur_node.is_terminal:
            break

    # Max depth exceeded, no path found
    else:
        return None

    return path


In [74]:
"""
planning script
"""

'\nplanning script\n'

In [17]:
import json
import molvs
from numpy import random
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch import optim
# import new_policies
from tqdm import tqdm
# from mcts import Node, mcts
from rdkit import Chem
from rdkit.Chem import AllChem
import ipdb
# print(ipdb.set_trace())



In [18]:
print('loading rules:::')
rollout_ruless = []
with open('data/rollout.dat', 'r') as f:
    for i, l in tqdm(enumerate(f), desc='rollout'):
        rule = l.strip()
        rollout_ruless.append(rule)
expansion_ruless = []
with open('data/expansion.dat', 'r') as f:
    for i, l in tqdm(enumerate(f), desc='expansion'):
        rule = l.strip()
        expansion_ruless.append(rule)

rollout: 1415it [00:00, 371863.42it/s]
expansion: 43637it [00:00, 278027.00it/s]

loading rules:::


In [19]:
starting_mols = set()
i=0
with open('data/emolecules.smi', 'r') as f:
    for line in tqdm(f, desc='Loading base compounds'):
        try:
            smi = line.strip()
            smi = molvs.standardize_smiles(smi)
            starting_mols.add(smi)
            i+=1
            if i>5000:
                break
        except Exception as e:
            pass
print('Base compounds:', len(starting_mols))
# print('Starting_mols:::',starting_mols)

Loading base compounds: 4991it [00:18, 219.16it/s]

Base compounds: 5001


In [20]:
rollout_net = RolloutPolicyNet(n_rules=len(rollout_rules))



In [22]:
expansion_net = ExpansionPolicyNet(idx,n_rules=len(expansion_rules))


Loading base compounds: 4991it [00:29, 219.16it/s]

In [21]:
filter_net = InScopeFilterNet()

In [ ]:
#load weights

In [32]:
def transform(mol, rule):
    rule = '[Br:1]-[C:2]-[C:3]-[#8:4]-[#16:7](=[O:8])(=[O:9])-[C:6]-[C&H3:5]>>[Br:1]-[C:2]-[C:3]-[O&H1:4].[C&H3:5]-[C:6]-[#16:7](=[O:8])=[O:9]'
    """Apply transformation rule to a molecule to get reactants"""
    """apply the reaction to a sequence of reactant molecules and return the products as a tuple of tuples"""
    rxn = AllChem.ReactionFromSmarts(rule)
    
    mol.UpdatePropertyCache(strict=False)
    mol=Chem.AddHs(mol, addCoords=True)

    print('check rxn:::',rxn,'typeofmol',type(mol),'typeofrxn::',type(rxn))
#     mol_smi = Chem.MolToSmiles(mol)
#     print('mol_smi',mol)

#     mol = Chem.MolFromSmiles(mol)
#     ps = rxn.RunReactants((m1,))
    results = rxn.RunReactants((mol, ))
    print('printing_results:::',results)

    # Only look at first set of results (TODO any reason not to?)
    results = results[0]
    reactants = [Chem.MolToSmiles(smi) for smi in results]
    return reactants

In [68]:
a  = 'bababaabab'
print(type(a))
b = {a}

<class 'str'>
<class 'set'>


In [75]:
def expansions(node):
    """Try expanding each molecule in the current state
    to possible reactants"""

    # Assume each mol is a SMILES string
    mols = node.state
    print('mols before condition:::',mols)


    # Convert mols to format for prediction
    # If the mol is in the starting set, ignore
    mols = [mol for mol in mols if mol not in starting_mols]
    print('mols in expansions:::',mols)
    fprs = fingerprint_mols(mols,fp_dim)
    fprs = np.array(fprs)
    fprs = torch.from_numpy(fprs).float()

    # Predict applicable rules
    expansion.eval()
    out = expansion(fprs)
    preds, top_labs = out.topk(5)
    print('expansion_preds5::',preds)
    print('expansion_labs5::',top_labs)
#     preds = sess.run(expansion_net.pred_op, feed_dict={
#         expansion_net.keep_prob: 1.,
#         expansion_net.X: fprs,
#         expansion_net.k: 5
#     })

    # Generate children for reactants
    children = []
#     for mol, rule_idxs in zip(mols, preds):
    for mol, rule_idxs in zip(mols, top_labs):
        # State for children will
        # not include this mol
        print('IN EXPANSION TYPE OF MOL:::',type(mol))
        mols = set(mols)
        mol_s = {mol}
        print('typpppp::',type(mol))
        new_state = mols - mol_s
#         new_state = mols

#         new_state = mols.remove(mol)
#         new_state = set()#added
        print('new_state::',new_state,'typeofmols in expansion::',type(mols))
        print('rule_idxs;:::',rule_idxs)
        mol = Chem.MolFromSmiles(mol,sanitize=False)
        for idx in rule_idxs:
            # Extract actual rule
            print('idx::',idx)
            print('typeof:::',type(expansion_ruless))
            rule = expansion_ruless[idx]

            # TODO filter_net should check if the reaction will work?
            # should do as a batch

            # Apply rule
            print('molecule,rule',mol,">>>><<<<<",rule)
            reactants = transform(mol, rule)

            if not reactants: continue

            state = new_state | set(reactants)
            print('state:::',state)
            terminal = all(mol in starting_mols for mol in state)
            child = Node(state=state, is_terminal=terminal, parent=node, action=rule)
            children.append(child)
    return children


In [63]:
def rollouts(node, max_depth=200):
    fp_dim = 8192
    cur = node
    for _ in range(max_depth):
        if cur.is_terminal:
            break

        # Select a random mol (that's not a starting mol)
        mols = [mol for mol in cur.state if mol not in starting_mols]
        print('mols::',mols)
        mol = random.choice(mols)
        print('mol in rollouts:::',mol,'<<<<>>>>>',mol)
        fprs = fingerprint_mols([mol],fp_dim)
        fprs = np.array(fprs)
        fprs = torch.from_numpy(fprs).float()

        # Predict applicable rules
        rollout.eval()
        out = rollout(fprs)
        preds, top_labs = out.topk(1)
        print('rollout_preds[0][0]:::',preds[0][0],int(preds[0][0]))
        print('top_labs_rollouts',top_labs)
        
#         preds = sess.run(rollout_net.pred_op, feed_dict={
#             expansion_net.keep_prob: 1.,
#             expansion_net.X: fprs,
#             expansion_net.k: 1
#         })

#         rule = rollout_ruless[preds[0][0]]#changed
        rule = rollout_ruless[top_labs[0][0]]
        reactants = transform(Chem.MolFromSmiles(mol), rule)
        print('reactants in rollouts',reactants)
        state = cur.state | set(reactants)
        print('state in rollout::',state)
        # State for children will
        # not include this mol
        state = state - {mol}
#         state = state.remove(mol)

        terminal = all(mol in starting_mols for mol in state)
        cur = Node(state=state, is_terminal=terminal, parent=cur, action=rule)

    # Max depth exceeded
    else:
        print('Rollout reached max depth')

        # Partial reward if some starting molecules are found
        reward = sum(1 for mol in cur.state if mol in starting_mols)/len(cur.state)

        # Reward of -1 if no starting molecules are found
        if reward == 0:
            return -1.

        return reward

    # Reward of 1 if solution is found
    return 1.

In [26]:
def plan(target_mol):
    """Generate a synthesis plan for a target molecule (in SMILES form).
    If a path is found, returns a list of (action, state) tuples.
    If a path is not found, returns None."""
    root = Node(state={target_mol})

    path = mcts(root, expansions, rollouts, iterations=2000, max_depth=200)
    if path is None:
        print('No synthesis path found. Try increasing `iterations` or `max_depth`.')
    else:
        print('Path found:')
        path = [(n.action, n.state) for n in path[1:]]
    return path

In [76]:
if __name__ == '__main__':
#     target_mol = '[H][C@@]12OC3=C(O)C=CC4=C3[C@@]11CCN(C)[C@]([H])(C4)[C@]1([H])C=C[C@@H]2O'
    fp_dim = 1e4
#     target_mol = 'CC(=O)NC1=CC=C(O)C=C1'
    target_mol ='CCS(=O)(=O)OCCBr'
    root = Node(state={target_mol})
    print('root:-',root)
    path = plan(target_mol)
    print('path:-',path)
#     import ipdb; 
#     ipdb.set_trace()


root:- <__main__.Node object at 0x7f5369c1a780>
mols before condition::: {'CCS(=O)(=O)OCCBr'}
mols in expansions::: ['CCS(=O)(=O)OCCBr']


/home/baba/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


expansion_preds5:: tensor([[2.4944e-05, 2.4930e-05, 2.4857e-05, 2.4844e-05, 2.4762e-05]],
       grad_fn=<TopkBackward>)
expansion_labs5:: tensor([[  465, 38945, 23056, 18085, 29862]])
IN EXPANSION TYPE OF MOL::: <class 'str'>
typpppp:: <class 'str'>
new_state:: set() typeofmols in expansion:: <class 'set'>
rule_idxs;::: tensor([  465, 38945, 23056, 18085, 29862])
idx:: tensor(465)
typeof::: <class 'list'>
molecule,rule <rdkit.Chem.rdchem.Mol object at 0x7f5369c1f580> >>>><<<<< [Br:1]-[c:2]1:[c:3]:[c:4]:[c:5]:[c:6]:[c:7]:1-[#16:8]-[C:13]-[C:12](-[Cl:14])-[C:11]-[C:10]-[C&H3:9]>>[Br:1]-[c:2]1:[c:3]:[c:4]:[c:5]:[c:6]:[c:7]:1-[S&H1:8].[C&H3:9]-[C:10]-[C:11]-[C:12](-[C:13])-[Cl:14]
check rxn::: <rdkit.Chem.rdChemReactions.ChemicalReaction object at 0x7f5369c1fda0> typeofmol <class 'rdkit.Chem.rdchem.Mol'> typeofrxn:: <class 'rdkit.Chem.rdChemReactions.ChemicalReaction'>
printing_results::: ((<rdkit.Chem.rdchem.Mol object at 0x7f53ee33d5b0>, <rdkit.Chem.rdchem.Mol object at 0x7f5369c3b978>)


  0%|          | 0/2000 [00:00<?, ?it/s]

printing_results::: ((<rdkit.Chem.rdchem.Mol object at 0x7f53ee33d5b0>, <rdkit.Chem.rdchem.Mol object at 0x7f5369c3b920>), (<rdkit.Chem.rdchem.Mol object at 0x7f5369bf0450>, <rdkit.Chem.rdchem.Mol object at 0x7f5369bf0240>))
state::: {'[H]C([H])([SH](=O)=O)[CH3]([H])([H])[H]', '[H]C([H])(O)C([H])([H])Br'}
root.children:- [<__main__.Node object at 0x7f5369c3d550>, <__main__.Node object at 0x7f5369c2f2e8>, <__main__.Node object at 0x7f5369c3d828>, <__main__.Node object at 0x7f5369c3d5f8>, <__main__.Node object at 0x7f5369c3d860>]
mols:: ['CCS(=O)(=O)OCCBr']
mol in rollouts::: CCS(=O)(=O)OCCBr <<<<>>>>> CCS(=O)(=O)OCCBr


/home/baba/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


rollout_preds[0][0]::: tensor(0.0008, grad_fn=<SelectBackward>) 0
top_labs_rollouts tensor([[1154]])
check rxn::: <rdkit.Chem.rdChemReactions.ChemicalReaction object at 0x7f5369c1f940> typeofmol <class 'rdkit.Chem.rdchem.Mol'> typeofrxn:: <class 'rdkit.Chem.rdChemReactions.ChemicalReaction'>
printing_results::: ((<rdkit.Chem.rdchem.Mol object at 0x7f5369c3b920>, <rdkit.Chem.rdchem.Mol object at 0x7f5369bf0240>), (<rdkit.Chem.rdchem.Mol object at 0x7f5369bf0558>, <rdkit.Chem.rdchem.Mol object at 0x7f5369bf0be0>))
reactants in rollouts ['[H]C([H])(O)C([H])([H])Br', '[H]C([H])([SH](=O)=O)[CH3]([H])([H])[H]']
state in rollout:: {'[H]C([H])(O)C([H])([H])Br', 'CCS(=O)(=O)OCCBr', '[H]C([H])([SH](=O)=O)[CH3]([H])([H])[H]'}
mols:: ['[H]C([H])([SH](=O)=O)[CH3]([H])([H])[H]', '[H]C([H])(O)C([H])([H])Br']
mol in rollouts::: [H]C([H])(O)C([H])([H])Br <<<<>>>>> [H]C([H])(O)C([H])([H])Br
rollout_preds[0][0]::: 

  0%|          | 0/2000 [00:01<?, ?it/s]

tensor(0.0008, grad_fn=<SelectBackward>) 0
top_labs_rollouts tensor([[1168]])
check rxn::: <rdkit.Chem.rdChemReactions.ChemicalReaction object at 0x7f5369c29cb0> typeofmol <class 'rdkit.Chem.rdchem.Mol'> typeofrxn:: <class 'rdkit.Chem.rdChemReactions.ChemicalReaction'>
printing_results::: ()


IndexError: tuple index out of range

In [5]:
rxn_s = '[Br:1]-[C:2]-[C:3]-[#8:4]-[#16:7](=[O:8])(=[O:9])-[C:6]-[C&H3:5]>>[Br:1]-[C:2]-[C:3]-[O&H1:4].[C&H3:5]-[C:6]-[#16:7](=[O:8])=[O:9]'
rxn = AllChem.ReactionFromSmarts(rxn_s)

rxn_no_h_s = '[O:1][C:2]>>[O:1]=[C:2]' 
rxn_no_h = AllChem.ReactionFromSmarts(rxn_no_h_s)


print(rxn)
print(rxn_no_h)



In [1]:
[Br:1]-[C:2]-[C:3]-[#8:4]-[#16:7](=[O:8])(=[O:9])-[C:6]-[C&H3:5]>>[Br:1]-[C:2]-[C:3]-[O&H1:4].[C&H3:5]-[C:6]-[#16:7](=[O:8])=[O:9] prod CCS(=O)(=O)OCCBr

SyntaxError: invalid syntax (<ipython-input-1-a6016ddb6fb3>, line 1)

In [18]:
meoh_1 = Chem.MolFromSmiles('CO')
meoh_2 = Chem.MolFromSmiles('CCS(=O)(=O)OCCBr')
meoh_3 = Chem.MolFromSmiles('CCS(=O)(=O)OCCBr', sanitize=False)
print(meoh_2)
print(Chem.MolToSmiles(meoh_1))
print(Chem.MolToSmiles(meoh_2))
print(Chem.MolToSmiles(meoh_3))


CO
CCS(=O)(=O)OCCBr
CCS(=O)(=O)OCCBr


In [10]:
print(rxn.RunReactants((meoh_1, )))
print(rxn.RunReactants((meoh_2, )))
# print(rxn.RunReactants((meoh_3, )))
reactants = rxn.RunReactants((meoh_2, ))

()
((<rdkit.Chem.rdchem.Mol object at 0x7f7ce813cb30>, <rdkit.Chem.rdchem.Mol object at 0x7f7ce813cfa8>), (<rdkit.Chem.rdchem.Mol object at 0x7f7ce813cbe0>, <rdkit.Chem.rdchem.Mol object at 0x7f7ce813c660>))


In [19]:
smi = Chem.MolToSmiles('<rdkit.Chem.rdchem.Mol object at 0x7f7ce813cb88>')
# print(Chem.MolToSmiles('<rdkit.Chem.rdchem.Mol object at 0x7f7ce813cb88>'))

ArgumentError: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(str)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)

In [50]:
print(rxn_no_h.RunReactants((meoh_1, )))
print(rxn_no_h.RunReactants((meoh_2, )))
print(rxn_no_h.RunReactants((meoh_3, )))

((<rdkit.Chem.rdchem.Mol object at 0x7f7cb3fb4ad8>,),)
((<rdkit.Chem.rdchem.Mol object at 0x7f7cb3fb4ad8>,),)
((<rdkit.Chem.rdchem.Mol object at 0x7f7cb3fb4ad8>,),)
